In [1]:
import numpy as np
import pandas as pd

In [2]:
datas = np.loadtxt("data/datingTestSet2.txt")
datas

array([[4.0920000e+04, 8.3269760e+00, 9.5395200e-01, 3.0000000e+00],
       [1.4488000e+04, 7.1534690e+00, 1.6739040e+00, 2.0000000e+00],
       [2.6052000e+04, 1.4418710e+00, 8.0512400e-01, 1.0000000e+00],
       ...,
       [2.6575000e+04, 1.0650102e+01, 8.6662700e-01, 3.0000000e+00],
       [4.8111000e+04, 9.1345280e+00, 7.2804500e-01, 3.0000000e+00],
       [4.3757000e+04, 7.8826010e+00, 1.3324460e+00, 3.0000000e+00]])

In [3]:
datas.shape

(1000, 4)

In [4]:
# df的每一列的数据类型可以不同
df = pd.read_csv("data/datingTestSet2.txt",  sep="\t", header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
0    1000 non-null int64
1    1000 non-null float64
2    1000 non-null float64
3    1000 non-null int64
dtypes: float64(2), int64(2)
memory usage: 31.3 KB


In [5]:
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,33635.421000,6.559961,0.832073,1.985000
std,21957.006833,4.243618,0.497239,0.818196
min,0.000000,0.000000,0.001156,1.000000
25%,13796.000000,2.933963,0.408995,1.000000
50%,31669.000000,6.595204,0.809420,2.000000
75%,47716.250000,10.056500,1.272847,3.000000
max,91273.000000,20.919349,1.695517,3.000000


In [6]:
# DataFrame转ndarray
x = df.iloc[:,0:-1].values
x.shape

(1000, 3)

In [7]:
y = df.iloc[:,-1].values
y.shape

(1000,)

In [8]:
x

array([[4.0920000e+04, 8.3269760e+00, 9.5395200e-01],
       [1.4488000e+04, 7.1534690e+00, 1.6739040e+00],
       [2.6052000e+04, 1.4418710e+00, 8.0512400e-01],
       ...,
       [2.6575000e+04, 1.0650102e+01, 8.6662700e-01],
       [4.8111000e+04, 9.1345280e+00, 7.2804500e-01],
       [4.3757000e+04, 7.8826010e+00, 1.3324460e+00]])

In [9]:
x.min(axis=0)

array([0.      , 0.      , 0.001156])

In [10]:
x.max(axis=0)

array([9.1273000e+04, 2.0919349e+01, 1.6955170e+00])

In [11]:
x.max(axis=0) - x.min(axis=0)

array([9.1273000e+04, 2.0919349e+01, 1.6943610e+00])

In [12]:
(x - x.min(axis=0)) / ( x.max(axis=0) - x.min(axis=0) )

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [13]:
def autoNorm(dataSet):
    """
    Desc:
        归一化特征值，消除特征之间量级不同导致的影响
    parameter:
        dataSet: 数据集
    return:
        归一化后的数据集 normDataSet. ranges和minVals即最小值与范围，并没有用到

    归一化公式：
        Y = (X-Xmin)/(Xmax-Xmin)
        其中的 min 和 max 分别是数据集中的最小特征值和最大特征值。该函数可以自动将数字特征值转化为0到1的区间。
    """
    # 计算每种属性的最大值、最小值、范围
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    # 极差
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    # 生成与最小值之差组成的矩阵
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    # 将最小值之差除以范围组成矩阵
    normDataSet = normDataSet / np.tile(ranges, (m, 1))  # element wise divide
    return normDataSet, ranges, minVals

In [14]:
def autoNorm2(dataSet):
    # 直接利用数组的广播即可，不用补齐数组
    minVals = x.min(axis=0)
    maxVals = x.max(axis=0)
    ranges = maxVals - minVals
    normDataSet = (x - minVals) / ranges
    return normDataSet, ranges, minVals

In [15]:
# 可以看到，广播的算法更方便也好理解
autoNorm(x)[0] == autoNorm2(x)[0]

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       ...,
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [16]:
x_norm, ranges, minVals = autoNorm2(x)

In [17]:
numTest = int(x_norm.shape[0] * 0.1)
numTrain = x_norm.shape[0] - numTest
numTrain

900

In [18]:
# 注意，就是在这个写错了(没有用归一化的数据)，造成第一次写的与用sklearn测试的相差加到
"""
x_train, x_test = x[:numTrain,:], x[numTrain:,:]
y_train, y_test = y[:numTrain], y[numTrain:]
"""
x_train, x_test = x_norm[:numTrain,:], x_norm[numTrain:,:]
y_train, y_test = y[:numTrain], y[numTrain:]

In [19]:
import collections

In [20]:
errorCount = 0
rst_ys = []
for item_x,item_y in zip(x_test,y_test):
    square_sum = np.sum(np.square(x_train - item_x), axis=1)    
    distances = np.sqrt(square_sum)
    # 取距离最近的k个的索引，这里用的是几何距离。
    sortedIndexs = distances.argsort()[0:5]
    counter = collections.Counter(y_train[sortedIndexs])
    rst = sorted([item[::-1] for item in counter.items()], reverse=True)[0]
    rst_y =  rst[1]
    rst_ys.append(rst_y)
    #print("the classifier came back with: %d, the real answer is: %d" % (rst_y, item_y))
    if (rst_y != item_y):  
        errorCount += 1

print("the total error rate is: %f" % (errorCount / float(numTest)))
print(errorCount)
np.array(rst_ys)

the total error rate is: 0.050000
5


array([1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 2, 3, 1, 3,
       3, 2, 3, 2, 3, 3, 2, 2, 1, 2, 1, 3, 3, 1, 1, 1, 1, 3, 2, 1, 2, 1,
       1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 1, 3, 1, 2, 3, 1, 2, 2, 2,
       2, 3, 2, 3, 3, 1, 2, 1, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 2, 1, 3, 3, 3])

In [21]:
a = np.array([[1,2],[1,2]])
np.square(a)

array([[1, 4],
       [1, 4]])

In [22]:
np.square(a).sum?

In [ ]:
np.square(a).sum

In [23]:
np.sum(np.square(a), axis=1)

array([5, 5])

In [24]:
np.sqrt( np.sum(np.square(a), axis=1) )

array([2.23606798, 2.23606798])

In [25]:
names = ['a','b','b','c','d','b','a']
counter = collections.Counter(names)

In [26]:
counter

Counter({'a': 2, 'b': 3, 'c': 1, 'd': 1})

In [27]:
counter.items()

dict_items([('a', 2), ('b', 3), ('c', 1), ('d', 1)])

In [28]:
sorted(counter.items())

[('a', 2), ('b', 3), ('c', 1), ('d', 1)]

In [29]:
[item[::-1] for item in counter.items()]

[(2, 'a'), (3, 'b'), (1, 'c'), (1, 'd')]

In [30]:
sorted([item[::-1] for item in counter.items()], reverse=True)

[(3, 'b'), (2, 'a'), (1, 'd'), (1, 'c')]

In [31]:
y_test

array([1, 2, 1, 2, 2, 1, 1, 2, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3,
       3, 2, 3, 2, 3, 3, 2, 2, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 2, 1, 2, 1,
       1, 2, 2, 1, 1, 1, 3, 1, 1, 2, 3, 2, 2, 1, 3, 1, 2, 3, 1, 2, 2, 2,
       2, 3, 2, 3, 3, 1, 2, 1, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 2, 1, 3, 3, 3])

In [32]:
y_test[[1,2,2,7,8]]

array([2, 1, 1, 2, 3])

'b'